# PSS 3, Episode 3, Light Auto ML 🦙  ...
## An Started Model Using Light Auto ML and Multiple New Features...

<img src='https://lightautoml.readthedocs.io/en/latest/_images/LightAutoML_logo_big.png' width = 650>

### Notebook Goals
* Learn more about LAMA.
* Develop a competitive model adding more features and playing with the model parameters.
* Continue learning better aproaches in ML.

### Credits:
I used this Notebook as inspiration to start my analysis...

If you like my Code please check this one...
https://www.kaggle.com/code/cv13j0/ps-s3-e3-lightautoml-woe-encoding/edit/run/116989049

# Installing Requiered Libraries...

In [ ]:
%%capture
!pip3 install -U lightautoml
!pip3 install -U pandas # Install pandas, workoround to solve issues...

---

# Loading Model Libraries...

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing Auto ML Libraries...
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

# Import PyTorch...
import torch

import torch.nn as nn
import torch.nn.functional as F

In [ ]:
%%time
from pathlib import Path # Import OS path libraries
from sklearn.preprocessing import LabelEncoder # Encode things

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error as mae
from sklearn.neighbors import KNeighborsRegressor

import seaborn as sns
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from category_encoders import WOEEncoder

---

# Configuring the Notebook Paramaters

In [ ]:
%%time
# I like to disable my Notebook Warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
# Notebook Configuration...

# Amount of data we want to load into the Model...
DATA_ROWS = None
# Dataframe, the amount of rows and cols to visualize...
NROWS = 50
NCOLS = 15
# Main data location path...
BASE_PATH = '...'

SEED = 42

In [ ]:
%%time
# Configure notebook display settings to only use 2 decimal places, tables look nicer.
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', NCOLS) 
pd.set_option('display.max_rows', NROWS)

---

# Loading the Competition Datasets...

In [ ]:
%%time
# Load the CSV information into a Pandas DataFrame...
input_path = Path('/kaggle/input/playground-series-s3e3')

trn_df = pd.read_csv(input_path / 'train.csv')
tst_df = pd.read_csv(input_path / 'test.csv')

original = pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

submission = pd.read_csv(input_path / 'sample_submission.csv')

In [ ]:
original = original.rename(columns = {'EmployeeNumber': 'id'})
original['Attrition'] = (original['Attrition'] == 'Yes').astype(int)

original = original[trn_df.columns.tolist()]

In [ ]:
%%time
trn_df['Generated'] = 1
tst_df['Generated'] = 1
original['Generated'] = 0

trn_df.drop('id', axis = 1, inplace = True)
original.drop('id', axis = 1, inplace = True)

# original = original[original['Attrition'] == 1] # Only keep the Attrition == 1 for balancing purposes...

trn_df = pd.concat([trn_df, original], ignore_index = True)
tst_df.drop('id', axis = 1, inplace = True)

---

# Auxiliary Functions and Data-Preparation...

In [ ]:
%%time
def identify_neg_cols(df):
    '''
    Identify cols with negative values...
    '''
    negatives = []
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    tmp = df.select_dtypes(include = numerics)
    for col in tmp.columns:
        if tmp[col].min() < 0:
            negatives.append(col)
    return negatives

negative_cols = identify_neg_cols(trn_df)
print(negative_cols, '\n')

In [ ]:
def identify_outliers(df):
    '''
    Identify outliers and flag them...
    '''
    factor = 3.0
    outlier_cols = []
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    tmp = df.select_dtypes(include = numerics)
    tmp = tmp[tmp.columns[~tmp.columns.isin(['id','Attrition', 'Generated'])]]
    for col in tmp.columns:
        lower = tmp[col].quantile(0.25)
        upper = tmp[col].quantile(0.75)
        IQR = upper - lower
        max_value = upper + factor * IQR
        min_value = lower - factor * IQR
        
        print(f'The Variable Analyzed is: {col}, Min: {min_value}, Max: {max_value}')
        tmp[col + '_Outlier'] = np.where((tmp[col] > max_value) | (tmp[col] < min_value), 1, 0)
        outlier_cols.append(col + '_Outlier')
    df['Is_Outlier'] = tmp[outlier_cols].apply(np.sum, axis=1)
    return df

trn_df = identify_outliers(trn_df)
tst_df = identify_outliers(tst_df)

print('Outliers Identified:', trn_df[trn_df['Is_Outlier'] == 1].shape)

---

# Feature Engineering...

In [ ]:
# Creating features...

# Feature 1
def is_young(x):
    '''
    Binary age separation below age cutoff...
    '''
    if x <= 25:
        return 1
    else:
        return 0

# Feature 2
def young_and_underpaid(x):
    '''
    Calculate if the worker is underpay and younger...
    '''
    if x['Age'] <= 25 and x['DailyRate'] < 500:
        return 1
    else:
        return 0
    
# Feature 3
def work_life(x):
    '''
    Calculate Worklife + Stocks Levels
    '''
    return x['WorkLifeBalance'] + x['StockOptionLevel']
    

# Feature 4
def overtime_stock(x):
    '''
    Calculates a satisfaction score
    '''
    
    if x['OverTime'] == 'Yes':
        return (x['MonthlyIncome'] * (x['StockOptionLevel'] + 0.05) * x['JobSatisfaction']) / x['Age']
    else:
        return (x['MonthlyIncome'] * (x['StockOptionLevel'] + 1.05) * x['JobSatisfaction']) / x['Age']
    

# Feature 5 
def income_satisfaction(x):
    '''
    Calculates income and satisfaction combination
    '''
    return x['JobSatisfaction'] * x['MonthlyIncome']
    

# Feature 6   
def income_level_enviroment_satisfaction(x):
    '''
    Calculates income and satisfaction combination
    '''
    return x['EnvironmentSatisfaction']*x['JobSatisfaction'] * (x['MonthlyIncome']/x['JobLevel'])
    
    
# Feature 7
def calculate_experience_ratios(df):
    '''
    Calculates multiple experience ratios...
    '''
    df['Equivalent_Salary'] = df['MonthlyIncome'] / (df['TotalWorkingYears'] + 1)
    df['Working_Years'] = df['YearsAtCompany'] / (df['TotalWorkingYears'] + 1)
    df['Avg_Years_Per_Company'] = df['TotalWorkingYears'] / (df['NumCompaniesWorked'] + 1)
    df['Education_To_Salary'] = df['MonthlyIncome'] / (df['Education'] + 1)
    df['Manager_Familiarity'] = df['YearsWithCurrManager'] / (df['YearsAtCompany'] + 1)
    df['Performance_Fairness'] = df['PercentSalaryHike'] / df['PerformanceRating']
    return df

In [ ]:
# Calculating features for Train...
trn_df['Is_young'] = trn_df['Age'].apply(lambda x: is_young(x))
trn_df['Young_And_Underpaid'] = trn_df.apply(lambda x: young_and_underpaid(x), axis = 1)
trn_df['Worklife_Stock'] = trn_df.apply(lambda x: work_life(x), axis = 1)

trn_df['Overtime_Stock'] = trn_df.apply(lambda x: overtime_stock(x), axis = 1)
trn_df['Income_Satisfaction'] = trn_df.apply(lambda x: income_satisfaction(x), axis = 1)
trn_df['Income_Level_Environment_Satisfaction'] = trn_df.apply(lambda x: income_level_enviroment_satisfaction(x), axis = 1)


# Calculating features for Test...
tst_df['Is_young'] = tst_df['Age'].apply(lambda x: is_young(x))
tst_df['Young_And_Underpaid'] = tst_df.apply(lambda x: young_and_underpaid(x), axis = 1)
tst_df['Worklife_Stock'] = tst_df.apply(lambda x: work_life(x), axis = 1)

tst_df['Overtime_Stock'] = tst_df.apply(lambda x: overtime_stock(x), axis = 1)
tst_df['Income_Satisfaction'] = tst_df.apply(lambda x: income_satisfaction(x), axis = 1)
tst_df['Income_Level_Environment_Satisfaction'] = tst_df.apply(lambda x: income_level_enviroment_satisfaction(x), axis = 1)

trn_df = calculate_experience_ratios(trn_df)
tst_df = calculate_experience_ratios(tst_df)

---

# Data Pre-Processing Before Training...

In [ ]:
def remove_non_variance(df):
    '''
    Remove from the dataframe non-changing features...
    '''
    feat_to_drop = [feat for feat in df.columns if df[feat].nunique() == 1]
    print(f'Dropping {feat_to_drop}')
    return feat_to_drop

feat_to_remove = remove_non_variance(trn_df)
trn_df = trn_df.drop(columns = feat_to_remove)
tst_df = tst_df.drop(columns = feat_to_remove)

In [ ]:
%%time 
# Creating an Encoding function.

# Fixed list of categotical features...
#categ_feat_list = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']
#categ_feat = [feat for feat in trn_df.columns if feat in categ_feat_list]

# Dynamic list of categorical features...
cutoff = 20
categ_feat = [feat for feat in trn_df.columns if trn_df[feat].nunique() < cutoff and feat != 'Attrition']
numeric_feat = [feat for feat in trn_df.columns if feat not in categ_feat and feat != 'Attrition']

# trn_df[categ_feat] = trn_df[categ_feat].astype('str') # Using the fixed feature list...

In [ ]:
# features = ['Age', 
#             'BusinessTravel', 
#             'DailyRate', 
#             'Department', 
#             'DistanceFromHome',
#             'Education', 
#             'EducationField', 
#             'EnvironmentSatisfaction',
#             'Gender',
#             'HourlyRate',
#             'JobInvolvement',
#             'JobLevel',
#             'JobRole',
#             'JobSatisfaction',
#             'MaritalStatus',
#             'MonthlyIncome',
#             'MonthlyRate',
#             'NumCompaniesWorked',
#             'OverTime',
#             'PercentSalaryHike',
#             'PerformanceRating',
#             'RelationshipSatisfaction',
#             'StockOptionLevel',
#             'TotalWorkingYears',
#             'TrainingTimesLastYear',
#             'WorkLifeBalance',
#             'YearsAtCompany',
#             'YearsInCurrentRole',
#             'YearsSinceLastPromotion',
#             'YearsWithCurrManager',
#             'is_generated',
#             'is_young',
#             'young_and_underpaid',
#             'worklife_stock',
#             'income_satisfaction',
#             'income_level_environ_job_sat',
#             'overtime_stock'
#            ]

In [ ]:
categ_feat = ['BusinessTravel',
              'Department',
              'Education', 
              'EducationField', 
              'EnvironmentSatisfaction', 
              'Gender',
              'JobInvolvement', 
              'JobLevel', 
              'JobRole', 
              'JobSatisfaction', 
              'MaritalStatus',
              'NumCompaniesWorked', 
              'OverTime', 
              'PerformanceRating', 
              'RelationshipSatisfaction', 
              'StockOptionLevel', 
              'TotalWorkingYears', 
              'TrainingTimesLastYear', 
              'WorkLifeBalance', 
              'YearsAtCompany',
              'Is_young', 
              'Young_And_Underpaid'
             ]

In [ ]:
%%time
def create_onehot(trn_df, tst_df, list_of_var = categ_feat):
    '''
    Onehot encoding multiple features.... 
    '''
    trn_df['Is_train'] = 1
    tst_df['Is_train'] = 0
    df = pd.concat([trn_df, tst_df])
    df = pd.get_dummies(df, columns = categ_feat)
    
    trn_df = df[df['Is_train'] == 1]
    tst_df = df[df['Is_train'] == 0]
    
    trn_df = trn_df.drop(['Is_train'], axis = 1)
    tst_df = tst_df.drop(['Is_train'], axis = 1)
    
    return trn_df,tst_df

# Will not be used in this iteration...
# trn_df, tst_df = create_onehot(trn_df, tst_df, list_of_var = categ_feat)

In [ ]:
%%time
def feature_encoder(trn, tst, list_of_var = categ_feat):
    '''
    Encode multiple features...
    '''
    
    encoded_feat = []
    trn['Is_train'] = 1
    tst['Is_train'] = 0
        
    df = pd.concat([trn,tst])

    for var in list_of_var:
        encoder = LabelEncoder()
        df[var + '_enc'] = encoder.fit_transform(df[var])
        encoded_feat.append(var + '_enc')
    
    trn = df[df['Is_train'] == 1]
    tst = df[df['Is_train'] == 0]
    
    trn = trn.drop(list_of_var, axis=1)
    tst = tst.drop(list_of_var, axis=1)
    return trn, tst, encoded_feat

#trn_df, tst_df, enc_feat = feature_encoder(trn_df, tst_df, list_of_var = categ_feat)

In [ ]:
%%time
def adv_feature_encoder(trn, tst, list_of_var = categ_feat):
    '''
    Encode multiple features...
    '''
    
    encoded_feat = []
    trn['Is_train'] = 1
    tst['Is_train'] = 0
        
    #df = pd.concat([trn, tst], ignore_index = True)
    
    for var in list_of_var:
        print(f'Converting >>> {var} ...')
        encoder = WOEEncoder(drop_invariant = True, randomized = True, verbose = 0)
        encoder.fit(trn[var].astype('str'), trn['Attrition'])
        
        # trn[var + '_enc'] = encoder.transform(trn[var])
        # tst[var + '_enc'] = encoder.transform(tst[var])
        # encoded_feat.append(var + '_enc')
        
        
        trn[var] = encoder.transform(trn[var].astype('str'))
        tst[var] = encoder.transform(tst[var].astype('str'))
        encoded_feat.append(var)
        
    # trn = trn.drop(list_of_var, axis=1)
    # tst = tst.drop(list_of_var, axis=1)

    return trn, tst, encoded_feat

trn_df, tst_df, enc_feat = adv_feature_encoder(trn_df, tst_df, list_of_var = categ_feat)

---

# Feature Selection Before Training...

In [ ]:
%%time
# Preprocessing the Information for Training.
TARGET = 'Attrition'
features = [feat for feat in trn_df.columns if feat not in [TARGET, 'Is_train', 'id', 'Generated']]

In [ ]:
%%time
print(np.sort(features))

In [ ]:
%%time
print(np.sort(enc_feat))

---

# Scaling Model Features...

In [ ]:
%%time
def scale_features(train, test, features):
    '''
    Scale the features...
    '''
    scalar = StandardScaler()
    train[features] = scalar.fit_transform(train[features])
    test[features] = scalar.transform(test[features])
    return train, test

trn_df, tst_df = scale_features(trn_df, tst_df, features)

---

# Model Development and Training...

In [ ]:
%%time
lgb_params = {'num_iterations'   : 772,
              'max_depth'        : 3,
              'learning_rate'    : 0.0293466,
              'min_child_samples': 36, 
              'num_leaves'       : 128, 
              'colsample_bytree' : 0.80, 
              'subsample'        : 0.90, 
              'subsample_freq'   : 5, 
              'reg_lambda'       : 28,
              'seed'             : SEED,
              'objective'        : 'binary',
              'boosting_type'    : 'gbdt',
              'device'           : 'cpu', 
              'gpu_platform_id'  : 0,
              'gpu_device_id'    : 0,
              'n_jobs'           : -1,
              'metric'           : 'auc',
              'verbose'          : -1,
             }

In [ ]:
%%time
cb_params = {'num_boost_round': 1420,
             'depth': 3,
             'learning_rate': 0.04895188,
             'rsm': 0.5,
             'subsample': 0.931,
             'l2_leaf_reg': 69,
             'min_data_in_leaf': 20,
             'random_strength': 0.175,
             'random_seed': SEED,
             'use_best_model': True,
             'task_type': 'CPU',
             'bootstrap_type': 'Bernoulli',
             'grow_policy': 'SymmetricTree',
             'loss_function': 'Logloss',
             'eval_metric': 'AUC'
            }

In [ ]:
# lgb_params = {
# #     'metric': 'binary_logloss',
#     'metric': 'auc',
# #     'n_estimators': 10000,
#     'objective': 'binary',
#     'learning_rate': 0.02,
#     'min_child_samples': 150,
#     'reg_alpha': 3e-5,
#     'reg_lambda': 9e-2,
#     'num_leaves': 20,
#     'max_depth': 16,
#     'colsample_bytree': 0.8,
#     'subsample': 0.8,
#     'subsample_freq': 2,
#     'max_bin': 240
# }

# cb_params = {
#     'max_depth':6,
#     'max_ctr_complexity': 5,
#     'num_trees': 50000,
#     'od_wait': 500,
#     'od_type':'Iter', 
#     'learning_rate': 0.04,
#     'min_data_in_leaf': 3
# }

---

# Light Auto ML Configuration

In [ ]:
task = Task('binary', metric = 'auc')

N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = SEED
TEST_SIZE = 0.20
TIMEOUT = 60 * 60
TARGET_NAME = 'Attrition'

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)
# TabularUtilizedAutoML
# TabularAutoML
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned', 'cb', 'cb_tuned']]},
    lgb_params = {'default_params': lgb_params, 'freeze_defaults': True},
    cb_params = {'default_params': cb_params, 'freeze_defaults': True},
    #reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
    #general_params = {"use_algos": [all_models]},
    reader_params = {'n_jobs': N_THREADS}
)

In [ ]:
# ...
roles = {'target': TARGET_NAME}

---

# Light Auto ML Training...

In [ ]:
# ...
preds_tr = automl.fit_predict(trn_df, roles = roles, verbose = 1)

In [ ]:
# 0.8714690602300498 ... Baseline
# 0.8715048088695989 ... Using 1.5 IQR
# 0.8716365355873238 ... Using 3.0 IQR, Less Filtering...
# 0.8721769528908104 ... Using 3.0 IQR, Added Additional Features... Best Score >>> 
# 0.8721769528908104 ... Using 3.0 IQR, Added Additional Features + cb_tunned...
# 0.8769689344490699 ... Using 3.0 IQR, Added Additional Features + cb_tunned + nn + 10 folds...

---

# Model Interpretability...

In [ ]:
%%time

# Fast feature importances calculation
fast_fi = automl.get_feature_scores('fast')
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (15, 10), grid = True)

---

# Generating Model Predictions...

In [ ]:
preds = automl.predict(tst_df)
preds

In [ ]:
submission['Attrition'] = preds.data[:,0]
submission

---

# Creating a Submission File for Kaggle...

In [ ]:
submission.to_csv('submission.csv', index = False)

---

# NN Model...

In [ ]:
%%time
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add, BatchNormalization, Dropout, Concatenate

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import random

from sklearn.model_selection import KFold, StratifiedKFold 
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
import datetime
import math

In [ ]:
%%time
def nn_model():
    '''
    '''
    
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))
    
    x = Dense(1024, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(inputs)
    X = Dropout(0.1)
    x = BatchNormalization()(x)
    
    
    x = Dense(256, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    X = Dropout(0.1)
    x = BatchNormalization()(x)

    
    x = Dense(128, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    X = Dropout(0.1)
    x = BatchNormalization()(x)
    
    
    x = Dense(64, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    X = Dropout(0.1)
    x = BatchNormalization()(x)    


    x = Dense(1 , 
              #use_bias  = True, 
              #kernel_regularizer = tf.keras.regularizers.l2(30e-6),
              activation = 'sigmoid')(x)
    
    model = Model(inputs, x)
    
    return model

In [ ]:
%%time
architecture = nn_model()
architecture.summary()

In [ ]:
%%time
# Defining model parameters...
BATCH_SIZE         = 64
EPOCHS             = 512 
EPOCHS_COSINEDECAY = 512
DIAGRAMS           = True
USE_PLATEAU        = False
INFERENCE          = False
VERBOSE            = 0 
TARGET             = 'Attrition'

In [ ]:
%%time
# Defining model training function...
def fit_model(X_train, y_train, X_val, y_val, run = 0):
    '''
    '''
    lr_start = 0.2
    start_time = datetime.datetime.now()
    
    #scaler = StandardScaler()
    #scaler = RobustScaler()
    #scaler = MinMaxScaler()
    #numerical = numeric_feat
    #X_train[numerical] = scaler.fit_transform(X_train[numerical])

    epochs = EPOCHS    
    lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 8, verbose = VERBOSE)
    es = EarlyStopping(monitor = 'val_loss', patience = 16, verbose = 1, mode = 'min', restore_best_weights = True)
    tm = tf.keras.callbacks.TerminateOnNaN()
    callbacks = [lr, es, tm]
    
    # Cosine Learning Rate Decay
    if USE_PLATEAU == False:
        epochs = EPOCHS_COSINEDECAY
        lr_end = 0.0002

        def cosine_decay(epoch):
            if epochs > 1:
                w = (1 + math.cos(epoch / (epochs - 1) * math.pi)) / 2
            else:
                w = 1
            return w * lr_start + (1 - w) * lr_end
        
        lr = LearningRateScheduler(cosine_decay, verbose = 0)
        callbacks = [lr, tm]
        
    model = nn_model()
    
    optimizer_func = tf.keras.optimizers.Adam(learning_rate = lr_start)
    loss_func = tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer = optimizer_func, loss = loss_func)
    
    #X_val[numerical] = scaler.transform(X_val[numerical])
    validation_data = (X_val, y_val)
    
    history = model.fit(X_train, 
                        y_train, 
                        validation_data = validation_data, 
                        epochs          = epochs,
                        verbose         = VERBOSE,
                        batch_size      = BATCH_SIZE,
                        shuffle         = True,
                        callbacks       = callbacks
                       )
    
    history_list.append(history.history)
    print(f'Training loss:{history_list[-1]["loss"][-1]:.3f}')
    callbacks, es, lr, tm, history = None, None, None, None, None
    
    
    y_val_pred = model.predict(X_val, batch_size = BATCH_SIZE, verbose = VERBOSE)
    
    score = roc_auc_score(y_val, y_val_pred)
    print(f'Fold {run}.{fold} | {str(datetime.datetime.now() - start_time)[-12:-7]}'
          f'| AUC: {score:.5f}')
    
    score_list.append(score)
    
    X_test = tst_df.copy()
    X_test = X_test[features]
    #X_test[numerical] = scaler.transform(X_test[numerical])
    tst_pred = model.predict(X_test)
    predictions.append(tst_pred)
    
    return model

In [ ]:
%%time
# Create empty lists to store NN information...
history_list = []
score_list   = []
predictions  = []

# Define kfolds for training purposes...
#kf = KFold(n_splits = 5)
kf = StratifiedKFold(n_splits = 10, random_state = SEED, shuffle = True)

for fold, (trn_idx, val_idx) in enumerate(kf.split(trn_df[features], trn_df[TARGET])):
    X_train, X_val = trn_df.iloc[trn_idx][features], trn_df.iloc[val_idx][features]
    y_train, y_val = trn_df.iloc[trn_idx][TARGET], trn_df.iloc[val_idx][TARGET]
    
    fit_model(X_train, y_train, X_val, y_val)
    
print('.'* 15, '\n')
print(f'OOF AUC: {np.mean(score_list):.5f}')

In [ ]:
%%time
# Populated the prediction on the submission dataset and creates an output file
nn_preds = np.array(predictions).mean(axis=0)
submission['Attrition'] = nn_preds
submission.to_csv('nn_submission.csv', index = False)